# Jupyter Notebook for mainnet deployment



### Necessary packages 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [ ]:
network = "mainnet"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [ ]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"




@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




### Required service

In [ ]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [ ]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


In [ ]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

In [ ]:
contracts_file_custom = "shubh_yeouido_contracts_20210112084238.pkl"

In [ ]:
with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

# Deploying all contracts


In [ ]:

contracts = {}
deploy = ['addressProvider', 'daoFund','delegation','governance', 'lendingPool', 'lendingPoolCore','lendingPoolDataProvider','liquidationManager','ommToken', 'priceOracle','rewards' ,'snapshot','worker_token']

directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "ommToken":
        params = {'_initialSupply':0, '_decimals': 18}
    elif item == "worker_token":
        params = {'_initialSupply':100, '_decimals': 18}
        
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(res)
    contracts[item] = res.get('scoreAddress', '')

    

In [ ]:
pprint(contracts)

## Deploying remaining contracts


In [ ]:
for item in deploy:
    if item not in contracts.keys():
        params = {}
        if item == "ommToken":
            params = {'_initialSupply':0, '_decimals': 18}
        elif item == "worker_token":
            params = {'_initialSupply':100, '_decimals': 18}
       
        deploy_transaction = DeployTransactionBuilder()\
            .from_(deployer_wallet.get_address())\
            .to(GOVERNANCE_ADDRESS)\
            .nid(NID)\
            .nonce(100)\
            .content_type("application/zip")\
            .content(gen_deploy_data_content(item))\
            .params(params)\
            .build()

        step_limit = icon_service.estimate_step(deploy_transaction) + 100000
        
        signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
        tx_hash = icon_service.send_transaction(signed_transaction)

        res = get_tx_result(tx_hash)
        print(res)
        contracts[item] = res.get('scoreAddress', '')

## Deploying otoken for usdb

In [ ]:
#deploy oUSDb
params = {"_name":"OmmUSDbInterestToken","_symbol":"oUSDb"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oUSDb'] = res.get('scoreAddress', '')


contracts

## Deploying otoken for icx

In [ ]:
#deploy oICX 
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

contracts

## Deploying otoken for iusdc


In [ ]:
#deploy oIUSDC
params = {"_name":"IconUSDInterest","_symbol":"oIUSDC","_decimals":6}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oIUSDC'] = res.get('scoreAddress', '')

contracts

## Other Contract Addresses

In [ ]:
# To Do: Assign mainnet contracts
contracts['sicx']="cxd59e42181ebcc5ab6c9e0803246d1e5170a7aff4"
contracts['staking']="cx634ac19ceaa647bbb06096fbc1e1337adb09361f"
contracts['iusdc']="cx65f639254090820361da483df233f6d0e69af9b7"
contracts['usdb']="cxaa068556df80f9917ef146e889f0b2c4b13ab634"


In [ ]:
pprint(contracts)

## Saving the pkl file


In [ ]:
now = datetime.utcnow()
filename =network+f'_contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'

print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:

    pkl.dump(contracts, f)

## Updating contracts

In [ ]:
# Updating mainnet contracts
contract_ = 'delegation'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
elif contract_ == 'oUSDb':
    update = contracts['oUSDb']
    contract_ = 'oToken'
elif contract_ == 'oIUSDC':
    update = contracts['oIUSDC']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

# Configuring SCOREs's parameters

In [ ]:
settings_lendingPool = [{'contract': 'lendingPool', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
                        {'contract': 'lendingPool', 'method': 'setLendingPoolDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
                        {'contract': 'lendingPool', 'method': 'setDaoFund', 'params':{'_address': contracts['daoFund']}},
                        {'contract': 'lendingPool', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
                        {'contract': 'lendingPool', 'method': 'setStaking', 'params':{'_address': contracts['staking']}},
                        {'contract': 'lendingPool', 'method': 'setOICX', 'params':{'_address': contracts['oICX']}},
                        {'contract': 'lendingPool', 'method': 'setReward','params': {'_address':contracts['rewards']}},
                        {'contract': 'lendingPool', 'method': 'setSICX','params': {'_address':contracts['sicx']}},
                        {'contract': 'lendingPool', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}}
                        {'contract': 'lendingPool', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}}
                       ]
for sett in settings_lendingPool:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))

In [ ]:
settings_lendinPoolDataProvider=[{'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['usdb'],'_sym':"USDB"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"USDC"}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCore', 'params':{'_address': contracts['lendingPoolCore']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setPriceOracle', 'params':{'_address': contracts['priceOracle']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPool', 'params':{'_address': contracts['lendingPool']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationManager', 'params':{'_address': contracts['liquidationManager']}},
                                {'contract': 'lendingPoolDataProvider', 'method': 'setStaking', 'params':{'_address': contracts['staking']}}
            ]
for sett in settings_lendinPoolDataProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
settings_oToken =[{'contract': 'oUSDb', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oUSDb', 'method': 'setReserve', 'params':{'_address':contracts['usdb']}},
            {'contract': 'oUSDb', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oUSDb', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'oUSDb', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}}]
for sett in settings_oToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
setting_priceOracle =[{'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
                      {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':10*10**17}},
                      {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'IUSDC','_quote':'USDC','_rate':10*10**17}}

                      ]
for sett in setting_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
setting_addressProvider =[  {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['usdb']}},
                            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oUSDb']}},
                            {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
                            {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
                            {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}},
                            {'contract': 'addressProvider', 'method': 'setIUSDC', 'params':{'_address':contracts['iusdc']}},
                            {'contract': 'addressProvider', 'method': 'setoIUSDC', 'params':{'_address':contracts['oIUSDC']}},
                            {'contract': 'addressProvider', 'method': 'setOmmToken', 'params':{'_address':contracts['ommToken']}},
                            {'contract': 'addressProvider', 'method': 'setDelegation', 'params':{'_address':contracts['delegation']}},
                            {'contract': 'addressProvider', 'method': 'setRewards', 'params':{'_address':contracts['rewards']}}
                         ]
for sett in setting_addressProvider:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
settings_reserves = [{'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['usdb'],
                                                   "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}},
                     {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['sicx'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                    "baseBorrowRate":f"0{'0'*17}",
                                                    "slopeRate1":f"8{'0'*16}",
                                                    "slopeRate2":f"2{'0'*18}"} ]}},
                    {'contract': 'lendingPoolCore', 
                        'method': 'setReserveConstants', 
                        'params' :{"_constants": [{"reserve":contracts['iusdc'],
                                                    "optimalUtilizationRate":f"8{'0'*17}",
                                                   "baseBorrowRate":f"2{'0'*16}",
                                                   "slopeRate1":f"6{'0'*16}",
                                                   "slopeRate2":f"1{'0'*18}"} ]}}
                    ]
for sett in settings_reserves:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
settings_oicx = [{'contract': 'oICX', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
                {'contract': 'oICX', 'method': 'setReserve', 'params':{'_address':contracts['sicx']}},
                {'contract': 'oICX', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                {'contract': 'oICX', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                {'contract': 'oICX', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}}]
for sett in settings_oicx:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
settings_oiusdc = [ {'contract': 'oIUSDC', 'method': 'setLendingPoolCore', 'params':{'_address':contracts['lendingPoolCore']}},
                    {'contract': 'oIUSDC', 'method': 'setReserve', 'params':{'_address':contracts['iusdc']}},
                    {'contract': 'oIUSDC', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
                    {'contract': 'oIUSDC', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
                    {'contract': 'oIUSDC', 'method': 'setLiquidation', 'params':{'_address':contracts['liquidationManager']}}
                    ]
for sett in settings_oiusdc:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
settings_liquidationManager =  [{'contract': 'liquidationManager', 'method': 'setLendingPoolDataProvider', 'params': {'_address':contracts['lendingPoolDataProvider']}},
                                {'contract': 'liquidationManager', 'method': 'setLendingPoolCore', 'params': {'_address':contracts['lendingPoolCore']}},
                                {'contract': 'liquidationManager', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
                                {'contract': 'liquidationManager', 'method': 'setFeeProvider','params': {'_address':contracts['feeProvider']}}
                                ]
for sett in settings_liquidationManager:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
settings_delegation =[{'contract': 'delegation', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
                     {'contract':  'delegation', 'method': 'setOmmToken','params': {'_address':contracts['ommToken']}}]
for sett in settings_delegation:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
settings_ommToken =[{'contract': 'ommToken', 'method': 'setAdmin','params': {'_admin':contracts['rewards']}},
                    {'contract':  'ommToken', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
                    {'contract':  'ommToken', 'method': 'setRewards','params': {'_address':contracts['rewards']}},
                    {'contract':  'ommToken', 'method': 'setMinimumStake','params': {'_min':10 * 10**18}},
                    {'contract':  'ommToken', 'method': 'setUnstakingPeriod','params': {'_time': 120}}]
for sett in settings_ommToken:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    print(get_tx_result(tx_hash))

In [ ]:
settings_lendingPoolCore =[ {'contract': 'lendingPoolCore', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
                            {'contract': 'lendingPoolCore', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
                            {'contract': 'lendingPoolCore', 'method': 'setPriceOracle','params': {'_address':contracts['priceOracle']}},
                            {'contract': 'lendingPoolCore', 'method': 'setLiquidationManager','params': {'_address':contracts['liquidationManager']}},
                            {'contract': 'lendingPoolCore', 'method': 'setDelegation','params': {'_address':contracts['delegation']}},
                            {'contract': 'lendingPoolCore', 'method': 'setStaking','params': {'_address':contracts['staking']}},
                            {'contract': 'lendingPoolCore', 'method': 'set_id','params': {'_value':'1'}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['usdb'],'_sym':"USDb"}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['sicx'],'_sym':"ICX"}},
                            {'contract': 'lendingPoolCore', 'method': 'setSymbol', 'params':{'_reserve': contracts['iusdc'],'_sym':"USDC"}}]
for sett in settings_lendingPoolCore:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

In [ ]:
settings_priceOracle =[{'contract':  'priceOracle', 'method': 'setBandOracle','params': {'_address':"cx61a36e5d10412e03c907a507d1e8c6c3856d9964"}},
                       {'contract':  'priceOracle', 'method': 'toggleOraclePriceBool','params':{}}]
for sett in settings_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
pprint(contracts)

In [ ]:
settings_snapshot =[{'contract': 'snapshot', 'method': 'setAdmin','params': {'_address':contracts['lendingPool']}},
                    {'contract': 'snapshot', 'method': 'setGovernance','params': {'_address':contracts['governance']}}]
for sett in settings_snapshot:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

In [ ]:
To add percentage for recipients
To add offset for balanced dex ( requires SCORE changes)

In [ ]:
settings_rewards=[ {'contract': 'rewards', 'method': 'setLendingPool','params': {'_address':contracts['lendingPool']}},
            {'contract': 'rewards', 'method': 'setOmm','params': {'_address':contracts['ommToken']}},
            {'contract': 'rewards', 'method': 'setLendingPoolCore','params': {'_address':contracts['lendingPoolCore']}},
            {'contract': 'rewards', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
            {'contract': 'rewards', 'method': 'setWorkerToken','params': {'_address':contracts['worker_token']}},
            {'contract': 'rewards', 'method': 'setAdmin','params': {'_address':contracts['governance']}},
            {'contract': 'rewards', 'method': 'setDaoFund','params': {'_address':contracts['daoFund']}},
            {'contract': 'rewards', 'method': 'setLpToken','params': {'_address':"cx291dacbb875a94b364194a5febaac4e6318681f7"}},
                  
            ]
for sett in settings_rewards:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])

## Timestamp to be added 

In [ ]:
settings_governance =[{'contract': 'governance', 'method': 'setSnapshot','params': {'_address':contracts['snapshot']}},
                      {'contract': 'governance', 'method': 'setRewards','params': {'_address':contracts['rewards']}},
                      {'contract': 'governance', 'method': 'setStartTimestamp','params': {'_timestamp': }},
                      ]
for sett in settings_governance:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))

## Adding USDb reserve in LendingPoolCore



In [ ]:
params ={"_reserve": {"reserveAddress":contracts['usdb'],"oTokenAddress":contracts['oUSDb'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Adding ICX reserve

In [ ]:
params ={"_reserve": {"reserveAddress":contracts['sicx'],"oTokenAddress":contracts['oICX'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Adding IUSDC reserve

In [ ]:
params ={"_reserve": {"reserveAddress":contracts['iusdc'],"oTokenAddress":contracts['oIUSDC'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"6","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Testing deposit

In [ ]:
params = {"_amount": 5 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(5 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)
